<a href="https://colab.research.google.com/github/abhidsawant/BasicTextEditor/blob/main/Quicksort_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!apt-get update
!apt-get install -y mpich

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [17]:
%%writefile parallel_quicksort.c
#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>

void quicksort(int* arr, int left, int right) {
    if (left >= right) return;
    int pivot = arr[(left + right) / 2], i = left, j = right;
    while (i <= j) {
        while (arr[i] < pivot) i++;
        while (arr[j] > pivot) j--;
        if (i <= j) {
            int tmp = arr[i];
            arr[i++] = arr[j];
            arr[j--] = tmp;
        }
    }
    quicksort(arr, left, j);
    quicksort(arr, i, right);
}

// Merge two sorted arrays
int* merge(int* a, int na, int* b, int nb) {
    int* res = (int*)malloc((na + nb) * sizeof(int));
    int i = 0, j = 0, k = 0;
    while (i < na && j < nb)
        res[k++] = (a[i] < b[j]) ? a[i++] : b[j++];
    while (i < na) res[k++] = a[i++];
    while (j < nb) res[k++] = b[j++];
    return res;
}

int main(int argc, char* argv[]) {
    // Initialize MPI
    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Start timing
    double start_time = MPI_Wtime();

    int n, *data = NULL;
    if (rank == 0) {
        FILE* f = fopen(argv[1], "r");
        fscanf(f, "%d", &n);
        data = (int*)malloc(n * sizeof(int));
        for (int i = 0; i < n; i++) fscanf(f, "%d", &data[i]);
        fclose(f);
    }

    MPI_Bcast(&n, 1, MPI_INT, 0, MPI_COMM_WORLD);
    int chunk = (n + size - 1) / size;
    int* local = (int*)malloc(chunk * sizeof(int));
    MPI_Scatter(data, chunk, MPI_INT, local, chunk, MPI_INT, 0, MPI_COMM_WORLD);

    int local_size = (rank == size - 1) ? n - chunk * rank : chunk;
    quicksort(local, 0, local_size - 1);

    for (int step = 1; step < size; step *= 2) {
        if (rank % (2 * step) != 0) {
            MPI_Send(local, local_size, MPI_INT, rank - step, 0, MPI_COMM_WORLD);
            free(local);
            break;
        }
        if (rank + step < size) {
            int recv_size = (rank + step >= size) ? 0 :
                           ((rank + 2 * step <= size) ? chunk * step : n - chunk * (rank + step));
            if (recv_size > 0) {
                int* recv = (int*)malloc(recv_size * sizeof(int));
                MPI_Recv(recv, recv_size, MPI_INT, rank + step, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
                int* merged = merge(local, local_size, recv, recv_size);
                free(local); free(recv);
                local = merged;
                local_size += recv_size;
            }
        }
    }

    // End timing
    double end_time = MPI_Wtime();
    double execution_time = end_time - start_time;

    // Gather timings from all processes
    double *all_times = NULL;
    if (rank == 0) {
        all_times = (double*)malloc(size * sizeof(double));
    }
    MPI_Gather(&execution_time, 1, MPI_DOUBLE, all_times, 1, MPI_DOUBLE, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        FILE* out = fopen(argv[2], "w");
        fprintf(out, "%d\n", local_size);
        for (int i = 0; i < local_size; i++) fprintf(out, "%d ", local[i]);
        fclose(out);

        // Calculate max and average execution time
        double max_time = all_times[0];
        double sum_time = all_times[0];
        for (int i = 1; i < size; i++) {
            if (all_times[i] > max_time) max_time = all_times[i];
            sum_time += all_times[i];
        }
        double avg_time = sum_time / size;

        printf("Sorted output written to %s\n", argv[2]);
        printf("Execution time statistics:\n");
        printf("  Process 0: %.6f seconds\n", execution_time);
        for (int i = 1; i < size; i++) {
            printf("  Process %d: %.6f seconds\n", i, all_times[i]);
        }
        printf("  Maximum time: %.6f seconds\n", max_time);
        printf("  Average time: %.6f seconds\n", avg_time);
        printf("  Total time: %.6f seconds\n", end_time - start_time);

        free(all_times);
    }

    free(local);
    if (data) free(data);
    MPI_Finalize();
    return 0;
}

Overwriting parallel_quicksort.c


In [18]:
%%writefile input.txt
5000
97 62 12 43 86 34 75 92 50 18 66 41 89 23 57 31 70 45 88 14 63 39 81 26 58 9 71 37 94 20 52 33 76 42 87 15 64 48 93 24 67 32 83 46 90 19 53 35 77 49
98 27 56 38 84 47 91 25 59 40 78 44 95 21 55 36 72 29 69 8 68 28 79 11 61 30 74 13 82 22 60 7 99 16 73 10 85 17 80 51 96 5 54 3 65 6 100 1 2 4
82 47 63 19 34 88 53 71 25 96 42 59 17 84 38 75 12 50 91 29 67 5 79 21 46 98 33 58 13 87 41 54 9 92 37 62 15 83 45 70 24 69 7 95 30 56 11 89 35 72
20 64 3 97 31 57 18 85 40 78 23 68 8 93 26 52 16 99 43 60 14 81 36 73 10 86 44 77 22 66 4 90 27 55 2 94 39 74 6 80 28 51 1 76 32 65 100 49 48 61
39 75 16 57 92 28 63 9 48 81 34 69 12 52 85 41 76 23 60 97 30 64 11 49 82 35 70 14 53 86 42 77 25 61 98 31 65 13 50 83 37 72 18 55 88 44 79 21 58 95
33 67 15 54 87 43 78 26 62 99 32 66 10 51 84 40 73 20 59 96 29 68 4 47 80 36 71 19 56 89 45 93 24 38 100 22 46 2 27 94 1 74 6 90 3 17 7 91 5 8
56 89 21 74 35 68 10 43 95 27 80 41 83 14 59 2 76 38 91 23 63 5 47 99 32 77 18 50 86 39 82 15 58 1 72 33 92 25 67 9 54 98 30 81 16 49 88 44 94 24
78 13 55 97 29 71 11 48 90 37 93 22 66 4 51 85 40 73 19 62 96 28 75 12 53 87 46 84 20 65 7 45 79 34 70 36 64 3 31 60 6 17 42 26 100 52 69 61 57
26 65 14 49 82 37 70 18 53 94 30 75 13 52 87 43 76 21 58 95 25 68 8 47 81 36 71 17 54 93 42 77 23 63 100 29 66 11 48 85 40 74 24 61 96 33 69 15 59 98
31 78 19 55 90 45 80 28 67 9 46 84 39 72 16 51 89 44 79 20 57 92 41 73 6 50 86 32 97 38 64 12 22 56 2 34 99 27 62 10 35 88 3 91 4 7 1 5 60 83
44 76 18 55 87 32 69 15 50 91 29 68 14 51 83 39 74 21 58 94 27 70 12 45 86 41 78 23 62 97 35 73 20 57 93 33 72 16 52 85 42 77 19 56 95 30 71 17 53 90
26 67 9 48 81 40 75 22 59 96 24 65 11 49 92 36 79 28 61 98 34 75 13 54 87 38 94 25 66 10 47 82 43 80 31 60 100 37 99 46 84 7 88 5 89 3 6 1 8 4
52 84 23 63 98 31 73 19 58 92 36 78 15 55 87 41 74 22 61 96 34 69 11 46 82 39 75 24 65 100 30 72 12 49 83 38 77 18 57 95 33 70 14 51 90 42 79 27 64 97
26 66 9 47 86 43 80 20 59 93 35 71 17 54 88 40 76 29 62 99 28 67 10 45 85 32 94 21 60 91 37 78 16 56 81 41 25 48 5 13 2 7 1 68 3 89 4 50 6 53
66 11 53 87 38 76 20 58 93 34 71 16 57 92 35 72 19 54 89 36 77 22 59 96 39 74 14 49 84 40 75 26 61 99 31 68 13 52 85 42 78 24 63 97 30 73 15 56 91 37
79 27 62 98 33 70 18 51 86 44 80 28 67 10 47 82 45 81 29 69 12 48 90 41 77 25 64 9 46 83 32 94 21 60 95 43 89 17 55 3 8 1 6 2 4 7 5 88 50 65
78 17 55 90 39 73 13 48 81 40 75 21 58 94 42 77 26 63 97 31 68 15 54 89 32 70 12 45 86 41 79 24 69 100 37 74 19 57 92 34 72 18 53 91 27 65 10 51 85 44
80 28 66 98 30 67 11 49 84 35 71 14 50 88 46 78 23 62 96 29 69 16 56 93 38 76 20 59 95 33 72 22 61 9 47 83 43 99 36 77 25 60 6 52 82 2 5 1 7 3
39 75 16 57 92 28 63 9 48 81 34 69 12 52 85 41 76 23 60 97 30 64 11 49 82 35 70 14 53 86 42 77 25 61 98 31 65 13 50 83 37 72 18 55 88 44 79 21 58 95
33 67 15 54 87 43 78 26 62 99 32 66 10 51 84 40 73 20 59 96 29 68 4 47 80 36 71 19 56 89 45 93 24 38 100 22 46 2 27 94 1 74 6 90 3 17 7 91 5 8
97 62 12 43 86 34 75 92 50 18 66 41 89 23 57 31 70 45 88 14 63 39 81 26 58 9 71 37 94 20 52 33 76 42 87 15 64 48 93 24 67 32 83 46 90 19 53 35 77 49
98 27 56 38 84 47 91 25 59 40 78 44 95 21 55 36 72 29 69 8 68 28 79 11 61 30 74 13 82 22 60 7 99 16 73 10 85 17 80 51 96 5 54 3 65 6 100 1 2 4
82 47 63 19 34 88 53 71 25 96 42 59 17 84 38 75 12 50 91 29 67 5 79 21 46 98 33 58 13 87 41 54 9 92 37 62 15 83 45 70 24 69 7 95 30 56 11 89 35 72
20 64 3 97 31 57 18 85 40 78 23 68 8 93 26 52 16 99 43 60 14 81 36 73 10 86 44 77 22 66 4 90 27 55 2 94 39 74 6 80 28 51 1 76 32 65 100 49 48 61
39 75 16 57 92 28 63 9 48 81 34 69 12 52 85 41 76 23 60 97 30 64 11 49 82 35 70 14 53 86 42 77 25 61 98 31 65 13 50 83 37 72 18 55 88 44 79 21 58 95
33 67 15 54 87 43 78 26 62 99 32 66 10 51 84 40 73 20 59 96 29 68 4 47 80 36 71 19 56 89 45 93 24 38 100 22 46 2 27 94 1 74 6 90 3 17 7 91 5 8
56 89 21 74 35 68 10 43 95 27 80 41 83 14 59 2 76 38 91 23 63 5 47 99 32 77 18 50 86 39 82 15 58 1 72 33 92 25 67 9 54 98 30 81 16 49 88 44 94 24
78 13 55 97 29 71 11 48 90 37 93 22 66 4 51 85 40 73 19 62 96 28 75 12 53 87 46 84 20 65 7 45 79 34 70 36 64 3 31 60 6 17 42 26 100 52 69 61 57
26 65 14 49 82 37 70 18 53 94 30 75 13 52 87 43 76 21 58 95 25 68 8 47 81 36 71 17 54 93 42 77 23 63 100 29 66 11 48 85 40 74 24 61 96 33 69 15 59 98
31 78 19 55 90 45 80 28 67 9 46 84 39 72 16 51 89 44 79 20 57 92 41 73 6 50 86 32 97 38 64 12 22 56 2 34 99 27 62 10 35 88 3 91 4 7 1 5 60 83
44 76 18 55 87 32 69 15 50 91 29 68 14 51 83 39 74 21 58 94 27 70 12 45 86 41 78 23 62 97 35 73 20 57 93 33 72 16 52 85 42 77 19 56 95 30 71 17 53 90
26 67 9 48 81 40 75 22 59 96 24 65 11 49 92 36 79 28 61 98 34 75 13 54 87 38 94 25 66 10 47 82 43 80 31 60 100 37 99 46 84 7 88 5 89 3 6 1 8 4
52 84 23 63 98 31 73 19 58 92 36 78 15 55 87 41 74 22 61 96 34 69 11 46 82 39 75 24 65 100 30 72 12 49 83 38 77 18 57 95 33 70 14 51 90 42 79 27 64 97
26 66 9 47 86 43 80 20 59 93 35 71 17 54 88 40 76 29 62 99 28 67 10 45 85 32 94 21 60 91 37 78 16 56 81 41 25 48 5 13 2 7 1 68 3 89 4 50 6 53
66 11 53 87 38 76 20 58 93 34 71 16 57 92 35 72 19 54 89 36 77 22 59 96 39 74 14 49 84 40 75 26 61 99 31 68 13 52 85 42 78 24 63 97 30 73 15 56 91 37
79 27 62 98 33 70 18 51 86 44 80 28 67 10 47 82 45 81 29 69 12 48 90 41 77 25 64 9 46 83 32 94 21 60 95 43 89 17 55 3 8 1 6 2 4 7 5 88 50 65
78 17 55 90 39 73 13 48 81 40 75 21 58 94 42 77 26 63 97 31 68 15 54 89 32 70 12 45 86 41 79 24 69 100 37 74 19 57 92 34 72 18 53 91 27 65 10 51 85 44
80 28 66 98 30 67 11 49 84 35 71 14 50 88 46 78 23 62 96 29 69 16 56 93 38 76 20 59 95 33 72 22 61 9 47 83 43 99 36 77 25 60 6 52 82 2 5 1 7 3
39 75 16 57 92 28 63 9 48 81 34 69 12 52 85 41 76 23 60 97 30 64 11 49 82 35 70 14 53 86 42 77 25 61 98 31 65 13 50 83 37 72 18 55 88 44 79 21 58 95
33 67 15 54 87 43 78 26 62 99 32 66 10 51 84 40 73 20 59 96 29 68 4 47 80 36 71 19 56 89 45 93 24 38 100 22 46 2 27 94 1 74 6 90 3 17 7 91 5 8
97 62 12 43 86 34 75 92 50 18 66 41 89 23 57 31 70 45 88 14 63 39 81 26 58 9 71 37 94 20 52 33 76 42 87 15 64 48 93 24 67 32 83 46 90 19 53 35 77 49
98 27 56 38 84 47 91 25 59 40 78 44 95 21 55 36 72 29 69 8 68 28 79 11 61 30 74 13 82 22 60 7 99 16 73 10 85 17 80 51 96 5 54 3 65 6 100 1 2 4
82 47 63 19 34 88 53 71 25 96 42 59 17 84 38 75 12 50 91 29 67 5 79 21 46 98 33 58 13 87 41 54 9 92 37 62 15 83 45 70 24 69 7 95 30 56 11 89 35 72
20 64 3 97 31 57 18 85 40 78 23 68 8 93 26 52 16 99 43 60 14 81 36 73 10 86 44 77 22 66 4 90 27 55 2 94 39 74 6 80 28 51 1 76 32 65 100 49 48 61
39 75 16 57 92 28 63 9 48 81 34 69 12 52 85 41 76 23 60 97 30 64 11 49 82 35 70 14 53 86 42 77 25 61 98 31 65 13 50 83 37 72 18 55 88 44 79 21 58 95
33 67 15 54 87 43 78 26 62 99 32 66 10 51 84 40 73 20 59 96 29 68 4 47 80 36 71 19 56 89 45 93 24 38 100 22 46 2 27 94 1 74 6 90 3 17 7 91 5 8
56 89 21 74 35 68 10 43 95 27 80 41 83 14 59 2 76 38 91 23 63 5 47 99 32 77 18 50 86 39 82 15 58 1 72 33 92 25 67 9 54 98 30 81 16 49 88 44 94 24
78 13 55 97 29 71 11 48 90 37 93 22 66 4 51 85 40 73 19 62 96 28 75 12 53 87 46 84 20 65 7 45 79 34 70 36 64 3 31 60 6 17 42 26 100 52 69 61 57
26 65 14 49 82 37 70 18 53 94 30 75 13 52 87 43 76 21 58 95 25 68 8 47 81 36 71 17 54 93 42 77 23 63 100 29 66 11 48 85 40 74 24 61 96 33 69 15 59 98
31 78 19 55 90 45 80 28 67 9 46 84 39 72 16 51 89 44 79 20 57 92 41 73 6 50 86 32 97 38 64 12 22 56 2 34 99 27 62 10 35 88 3 91 4 7 1 5 60 83
44 76 18 55 87 32 69 15 50 91 29 68 14 51 83 39 74 21 58 94 27 70 12 45 86 41 78 23 62 97 35 73 20 57 93 33 72 16 52 85 42 77 19 56 95 30 71 17 53 90
26 67 9 48 81 40 75 22 59 96 24 65 11 49 92 36 79 28 61 98 34 75 13 54 87 38 94 25 66 10 47 82 43 80 31 60 100 37 99 46 84 7 88 5 89 3 6 1 8 4
52 84 23 63 98 31 73 19 58 92 36 78 15 55 87 41 74 22 61 96 34 69 11 46 82 39 75 24 65 100 30 72 12 49 83 38 77 18 57 95 33 70 14 51 90 42 79 27 64 97
26 66 9 47 86 43 80 20 59 93 35 71 17 54 88 40 76 29 62 99 28 67 10 45 85 32 94 21 60 91 37 78 16 56 81 41 25 48 5 13 2 7 1 68 3 89 4 50 6 53
66 11 53 87 38 76 20 58 93 34 71 16 57 92 35 72 19 54 89 36 77 22 59 96 39 74 14 49 84 40 75 26 61 99 31 68 13 52 85 42 78 24 63 97 30 73 15 56 91 37
79 27 62 98 33 70 18 51 86 44 80 28 67 10 47 82 45 81 29 69 12 48 90 41 77 25 64 9 46 83 32 94 21 60 95 43 89 17 55 3 8 1 6 2 4 7 5 88 50 65
78 17 55 90 39 73 13 48 81 40 75 21 58 94 42 77 26 63 97 31 68 15 54 89 32 70 12 45 86 41 79 24 69 100 37 74 19 57 92 34 72 18 53 91 27 65 10 51 85 44
80 28 66 98 30 67 11 49 84 35 71 14 50 88 46 78 23 62 96 29 69 16 56 93 38 76 20 59 95 33 72 22 61 9 47 83 43 99 36 77 25 60 6 52 82 2 5 1 7 3
39 75 16 57 92 28 63 9 48 81 34 69 12 52 85 41 76 23 60 97 30 64 11 49 82 35 70 14 53 86 42 77 25 61 98 31 65 13 50 83 37 72 18 55 88 44 79 21 58 95
33 67 15 54 87 43 78 26 62 99 32 66 10 51 84 40 73 20 59 96 29 68 4 47 80 36 71 19 56 89 45 93 24 38 100 22 46 2 27 94 1 74 6 90 3 17 7 91 5 8
97 62 12 43 86 34 75 92 50 18 66 41 89 23 57 31 70 45 88 14 63 39 81 26 58 9 71 37 94 20 52 33 76 42 87 15 64 48 93 24 67 32 83 46 90 19 53 35 77 49
98 27 56 38 84 47 91 25 59 40 78 44 95 21 55 36 72 29 69 8 68 28 79 11 61 30 74 13 82 22 60 7 99 16 73 10 85 17 80 51 96 5 54 3 65 6 100 1 2 4
82 47 63 19 34 88 53 71 25 96 42 59 17 84 38 75 12 50 91 29 67 5 79 21 46 98 33 58 13 87 41 54 9 92 37 62 15 83 45 70 24 69 7 95 30 56 11 89 35 72
20 64 3 97 31 57 18 85 40 78 23 68 8 93 26 52 16 99 43 60 14 81 36 73 10 86 44 77 22 66 4 90 27 55 2 94 39 74 6 80 28 51 1 76 32 65 100 49 48 61
39 75 16 57 92 28 63 9 48 81 34 69 12 52 85 41 76 23 60 97 30 64 11 49 82 35 70 14 53 86 42 77 25 61 98 31 65 13 50 83 37 72 18 55 88 44 79 21 58 95
33 67 15 54 87 43 78 26 62 99 32 66 10 51 84 40 73 20 59 96 29 68 4 47 80 36 71 19 56 89 45 93 24 38 100 22 46 2 27 94 1 74 6 90 3 17 7 91 5 8
56 89 21 74 35 68 10 43 95 27 80 41 83 14 59 2 76 38 91 23 63 5 47 99 32 77 18 50 86 39 82 15 58 1 72 33 92 25 67 9 54 98 30 81 16 49 88 44 94 24
78 13 55 97 29 71 11 48 90 37 93 22 66 4 51 85 40 73 19 62 96 28 75 12 53 87 46 84 20 65 7 45 79 34 70 36 64 3 31 60 6 17 42 26 100 52 69 61 57
26 65 14 49 82 37 70 18 53 94 30 75 13 52 87 43 76 21 58 95 25 68 8 47 81 36 71 17 54 93 42 77 23 63 100 29 66 11 48 85 40 74 24 61 96 33 69 15 59 98
31 78 19 55 90 45 80 28 67 9 46 84 39 72 16 51 89 44 79 20 57 92 41 73 6 50 86 32 97 38 64 12 22 56 2 34 99 27 62 10 35 88 3 91 4 7 1 5 60 83
44 76 18 55 87 32 69 15 50 91 29 68 14 51 83 39 74 21 58 94 27 70 12 45 86 41 78 23 62 97 35 73 20 57 93 33 72 16 52 85 42 77 19 56 95 30 71 17 53 90
26 67 9 48 81 40 75 22 59 96 24 65 11 49 92 36 79 28 61 98 34 75 13 54 87 38 94 25 66 10 47 82 43 80 31 60 100 37 99 46 84 7 88 5 89 3 6 1 8 4
52 84 23 63 98 31 73 19 58 92 36 78 15 55 87 41 74 22 61 96 34 69 11 46 82 39 75 24 65 100 30 72 12 49 83 38 77 18 57 95 33 70 14 51 90 42 79 27 64 97
26 66 9 47 86 43 80 20 59 93 35 71 17 54 88 40 76 29 62 99 28 67 10 45 85 32 94 21 60 91 37 78 16 56 81 41 25 48 5 13 2 7 1 68 3 89 4 50 6 53
66 11 53 87 38 76 20 58 93 34 71 16 57 92 35 72 19 54 89 36 77 22 59 96 39 74 14 49 84 40 75 26 61 99 31 68 13 52 85 42 78 24 63 97 30 73 15 56 91 37
79 27 62 98 33 70 18 51 86 44 80 28 67 10 47 82 45 81 29 69 12 48 90 41 77 25 64 9 46 83 32 94 21 60 95 43 89 17 55 3 8 1 6 2 4 7 5 88 50 65
78 17 55 90 39 73 13 48 81 40 75 21 58 94 42 77 26 63 97 31 68 15 54 89 32 70 12 45 86 41 79 24 69 100 37 74 19 57 92 34 72 18 53 91 27 65 10 51 85 44
80 28 66 98 30 67 11 49 84 35 71 14 50 88 46 78 23 62 96 29 69 16 56 93 38 76 20 59 95 33 72 22 61 9 47 83 43 99 36 77 25 60 6 52 82 2 5 1 7 3

Overwriting input.txt


In [19]:
# Compile the code
!mpicc -o parallel_quicksort parallel_quicksort.c

In [20]:
# Try with just 1 process
!mpirun --allow-run-as-root -np 1 ./parallel_quicksort input.txt output.txt

Sorted output written to output.txt
Execution time statistics:
  Process 0: 0.000925 seconds
  Maximum time: 0.000925 seconds
  Average time: 0.000925 seconds
  Total time: 0.000925 seconds


In [21]:
# Check if the output file exists and view its contents
!ls -l output.txt
!cat output.txt

-rw-r--r-- 1 root root 13605 Apr 22 08:01 output.txt
5000
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 